# Resource References

This page is copied from https://hapifhir.io/doc_resource_references.html

As the original page doesn't show the version of hapi api is being used, there're several code snippets with inconsistent programming interface references. It causes some of the code not runnable or with compile errors. Thoses errors are corrected in this notebook. This demonstrates the value of Jupyter notebooks where you can verify the code to be valid in action. 

Resource references are a key part of the HAPI FHIR model, since almost any resource will have references to other resources within it.

The ResourceReferenceDt type is the datatype for references. This datatype has a number of properties which help make working with FHIR simple.

The getReference() method returns an IdDt instance which contains the identity of the resource being referenced. This is the item which is most commonly populated when interacting with FHIR. For example, consider the following Patient resource, which contains a reference to an Organization resource:

```xml
<Patient xmlns="http://hl7.org/fhir">
   <identifier>
      <system value="urn:mrns"/>
      <value value="253345"/>
   </identifier>
   <managingOrganization>
      <reference value="Organization/112"/>
   </managingOrganization>
</Patient>
```

Given a Patient resource obtained by invoking a client operation, a call to `IdDt ref = patient.getManagingOrganization().getReference();` returns an instance of IdDt which contains the "Organization/112" reference.

ResourceReferenceDt also has a field for storing actual resource instances however, and this can be very useful.

## Load required libraries

In [27]:
%%loadFromPOM
<dependency>
    <groupId>ca.uhn.hapi.fhir</groupId>
    <artifactId>hapi-fhir-structures-r4</artifactId>
    <version>4.0.3</version>
</dependency>

# References in Server Code

In server code, you will often want to return a resource which contains a link to another resource. Generally these "linked" resources are not actually included in the response, but rather a link to the resource is included and the client may request that resource directly (by ID) if it is needed.

The following example shows a Patient resource being created which will have a link to its managing organization when encoded from a server:



In [28]:
import ca.uhn.fhir.context.FhirContext;
FhirContext ctxR4 = FhirContext.forR4();

In [29]:
import org.hl7.fhir.r4.model.Patient;
Patient patient = new Patient();
patient.setId("Patient/1333");
patient.addIdentifier().setSystem("urn:mrns").setValue("253345");
patient.getManagingOrganization().setReference("Organization/124362");
String encoded = ctxR4.newXmlParser().setPrettyPrint(true).encodeResourceToString(patient);
System.out.println(encoded);

<Patient xmlns="http://hl7.org/fhir">
   <id value="1333"></id>
   <identifier>
      <system value="urn:mrns"></system>
      <value value="253345"></value>
   </identifier>
   <managingOrganization>
      <reference value="Organization/124362"></reference>
   </managingOrganization>
</Patient>


### Handling Includes (_include) in a Bundle

Your server code may also wish to add additional resource to a bundle being returned (e.g. because of an _include directive in the client's request).

To do this, you can implement your server method to simply return `List<IResource>` and then simply add your extra resources to the list. Another technique however, is to populate the reference as shown in the example below, but ensure that the referenced resource has an ID set.

In the following example, the Organization resource has an ID set, so it will not be contained but will rather appear as a distinct entry in any returned bundles. Both resources are added to a bundle, which will then have two entries:

In [42]:
import org.hl7.fhir.instance.model.api.IBaseResource;
import org.hl7.fhir.r4.model.Organization;
import org.hl7.fhir.r4.model.Bundle;
import org.hl7.fhir.r4.model.Bundle.BundleType;
import org.hl7.fhir.r4.model.Bundle.BundleEntryComponent;
import org.hl7.fhir.r4.model.Resource;
import ca.uhn.fhir.rest.annotation.Search;

@Search
private List<Resource> searchForPatients() {
   // Create an organization
   Organization org = new Organization();
   org.setId("Organization/65546");
   org.setName("Test Organization");
 
   // Create a patient
   Patient patient = new Patient();
   patient.setId("Patient/1333");
   patient.addIdentifier().setSystem("urn:mrns").setValue("253345");
   patient.getManagingOrganization().setResource(org);
 
   // Here we return only the patient object, which has links to other resources
   List<Resource> retVal = new ArrayList<Resource>();
   retVal.add(patient);
   return retVal;
}


### Contained Resources

On the other hand, if the linked resource does not have an ID set, the linked resource will be included in the returned bundle as a "contained" resource. In this case, HAPI itself will define a local reference ID (e.g. "#1").

In [31]:
// Create an organization, note that the organization does not have an ID
Organization org = new Organization();
org.setName("Contained Test Organization");
 
Patient patient = new Patient();
patient.setId("Patient/1333");
patient.addIdentifier().setSystem("urn:mrns").setValue("253345"); 
// Put the organization as a reference in the patient resource
patient.getManagingOrganization().setResource(org);
 
String encoded = ctxR4.newXmlParser().setPrettyPrint(true).encodeResourceToString(patient);
System.out.println(encoded);

<Patient xmlns="http://hl7.org/fhir">
   <id value="1333"></id>
   <contained>
      <Organization xmlns="http://hl7.org/fhir">
         <id value="1"></id>
         <name value="Contained Test Organization"></name>
      </Organization>
   </contained>
   <identifier>
      <system value="urn:mrns"></system>
      <value value="253345"></value>
   </identifier>
   <managingOrganization>
      <reference value="#1"></reference>
   </managingOrganization>
</Patient>


Note that you may also "contain" resources manually in your own code if you prefer. The following example show how to do this:

In [32]:
// Create an organization, and give it a local ID
Organization org = new Organization();
org.setId("#localOrganization");
org.getNameElement().setValue("Contained Test Organization");
 
// Create a patient
Patient patient = new Patient();
patient.setId("Patient/1333");
patient.addIdentifier().setSystem("urn:mrns").setValue("253345");
 
// Set the reference, and manually add the contained resource
patient.getManagingOrganization().setReference("#localOrganization");
patient.getContained().add(org);
 
String encoded = ctxR4.newXmlParser().setPrettyPrint(true).encodeResourceToString(patient);
System.out.println(encoded);

<Patient xmlns="http://hl7.org/fhir">
   <id value="1333"></id>
   <contained>
      <Organization xmlns="http://hl7.org/fhir">
         <id value="localOrganization"></id>
         <name value="Contained Test Organization"></name>
      </Organization>
   </contained>
   <identifier>
      <system value="urn:mrns"></system>
      <value value="253345"></value>
   </identifier>
   <managingOrganization>
      <reference value="#localOrganization"></reference>
   </managingOrganization>
</Patient>
